In [ ]:
# Sentinel-2 cloudless data download with memory optimization

# Importing libraries
import ee
import geemap
import os
import rasterio
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-gistg008')

# Set file paths and region of interest
shapefile = r"C:\Users\SuperAdmin\Desktop\TOPOSHEET\combodia.shp"
folder_path = r"C:\Users\SuperAdmin\Desktop\Automation_1m"
base_name = "S2_test_combodia_leaf_on"
region = geemap.shp_to_ee(shapefile).geometry()

# Function to split the region
def split_region(region, num_splits):
    coords = region.bounds().coordinates().getInfo()[0]
    min_x = min(coords, key=lambda x: x[0])[0]
    max_x = max(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_y = max(coords, key=lambda x: x[1])[1]

    width = (max_x - min_x) / num_splits
    height = (max_y - min_y) / num_splits
    
    regions = []
    for i in range(num_splits):
        for j in range(num_splits):
            xmin = min_x + i * width
            xmax = min_x + (i + 1) * width
            ymin = min_y + j * height
            ymax = min_y + (j + 1) * height
            regions.append(ee.Geometry.Rectangle([xmin, ymin, xmax, ymax]))

    return regions  

# Function to export image parts
def export_image_parts(image, regions, base_name, folder_path, scale=10, crs='EPSG:4326'):
    part_paths = []
    for idx, reg in enumerate(regions):
        part_name = f"{base_name}_{idx+1}.tif"
        part_path = os.path.join(folder_path, part_name)
        part_paths.append(part_path)
        try:
            geemap.ee_export_image(
                image,
                filename=part_path,
                region=reg,
                scale=scale,
                crs=crs
            )
            print(f"Exported {part_name}")
        except Exception as e:
            print(f"Failed to export {part_name}: {e}")
    return part_paths

# Function to check file size
def check_file_size(file_path):
    return os.path.getsize(file_path) if os.path.exists(file_path) else 0

# Function to find optimal number of splits
def find_optimal_splits(image, initial_splits, max_size_bytes, folder_path, base_name, scale=10, crs='EPSG:4326'):
    num_splits = initial_splits
    while True:
        regions = split_region(region, num_splits)
        part_paths = export_image_parts(image, regions, base_name, folder_path, scale, crs)
        
        # Check size of the first part
        first_part_path = part_paths[0]
        file_size = check_file_size(first_part_path)
        
        if file_size > 0 and file_size <= max_size_bytes:
            print(f"Optimal number of splits: {num_splits}")
            return part_paths
        else:
            # Clean up previously exported files before retrying
            for part_path in part_paths:
                if os.path.exists(part_path):
                    os.remove(part_path)
            # Increase splits and retry
            num_splits += 1

# Cloudless Sentinel-2 Image Collection
def get_s2_sr_cld_col(aoi, start_date, end_date):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(aoi)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)))
    s2_clouds_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                     .filterBounds(aoi)
                     .filterDate(start_date, end_date))
    joined = ee.Join.saveFirst('s2cloudless').apply(s2_sr_col, s2_clouds_col, ee.Filter.equals(
        leftField='system:index', rightField='system:index'
    ))
    return ee.ImageCollection(joined)

def add_cloud_bands(img):
    cloud_prob = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cloud_prob.gt(65).rename('clouds')
    return img.addBands(ee.Image([cloud_prob, is_cloud]))

def add_shadow_bands(img):
    not_water = img.select('SCL').neq(6)
    cloud_proj = (img.select('clouds')
                  .directionalDistanceTransform(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).add(180), 1)
                  .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
                  .select('distance')
                  .mask()
                  .rename('cloud_transform'))
    shadows = cloud_proj.multiply(not_water).rename('shadows')
    return img.addBands(ee.Image([cloud_proj, shadows]))

def add_cld_shdw_mask(img):
    img_cloud = add_cloud_bands(img)
    img_cloud_shadow = add_shadow_bands(img_cloud)
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    return img_cloud_shadow.addBands(is_cld_shdw.rename('cloudmask'))

def apply_cld_shdw_mask(img):
    masked_img = img.updateMask(img.select('cloudmask').Not()).divide(10000)
    return masked_img.select('B.*').copyProperties(img, ['system:time_start'])

# Parameters
start_date = '2023-12-01'
end_date = '2024-01-31'
max_size_bytes = 50331648  # Maximum size in bytes

# Get and process Sentinel-2 images
s2_sr_cld_col = get_s2_sr_cld_col(region, start_date, end_date)
s2_cloudless_col = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

# Mosaic and clip to the region of interest
s2_mosaic = s2_cloudless_col.median().clip(region)

# Find optimal number of splits and export images
part_paths = find_optimal_splits(
    s2_mosaic,
    initial_splits=1,
    max_size_bytes=max_size_bytes,
    folder_path=folder_path,
    base_name=base_name,
    scale=10,
    crs='EPSG:4326'
)

# Define paths for merging and clipping
def get_tiff_files(folder_path, base_name):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.startswith(base_name) and f.endswith('.tif')]

def merge_tiff_files(tiff_files, output_path):
    src_files_to_mosaic = [rasterio.open(f) for f in tiff_files]
    mosaic, transform = merge(src_files_to_mosaic)
    
    metadata = src_files_to_mosaic[0].meta.copy()
    metadata.update({
        'driver': 'GTiff',
        'count': mosaic.shape[0],
        'height': mosaic.shape[1],
        'width': mosaic.shape[2],
        'transform': transform,
        'nodata': 0
    })
    
    with rasterio.open(output_path, 'w', **metadata) as dest:
        dest.write(mosaic)
    
    for src in src_files_to_mosaic:
        src.close()

def clip_with_shapefile(tiff_path, shapefile_path, output_path):
    gdf = gpd.read_file(shapefile_path)
    
    # Reproject shapefile if necessary
    with rasterio.open(tiff_path) as src:
        if gdf.crs != src.crs:
            gdf = gdf.to_crs(src.crs)
        shapes = [feature['geometry'] for feature in gdf.iterfeatures()]
        
        nodata = src.nodata if src.nodata is not None else 0
        
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            'driver': 'GTiff',
            'height': out_image.shape[1],
            'width': out_image.shape[2],
            'transform': out_transform,
            'nodata': nodata
        })
        
        with rasterio.open(output_path, 'w', **out_meta) as dest:
            dest.write(out_image)

# Paths for merging and clipping
if part_paths:
    merged_tiff_path = os.path.join(folder_path, f"{base_name}__merged.tif")
    clipped_tiff_path = os.path.join(folder_path, f"{base_name}_clipped.tif")

    # Merge TIFF files
    merge_tiff_files(part_paths, merged_tiff_path)

    # Clip the merged TIFF
    clip_with_shapefile(merged_tiff_path, shapefile, clipped_tiff_path)

    # Clean up intermediate files
    for path in part_paths:
        if os.path.exists(path):
            os.remove(path)
    if os.path.exists(merged_tiff_path):
        os.remove(merged_tiff_path)

    print("Merged and clipped image saved successfully!")
else:
    print("No parts were exported.")